## Markov Chains

![markov_chain.jpg](images/markov_chain.jpg)
[Source](https://mb-14.github.io/tech/2018/10/24/gomarkov.html)

<br>

Markov chains are a simple way of **next value prediction**. This value may be for e.g. a next word, the next number or some next decision.

All (historic) information used to compute this next value lies in the current value. So the next value is not based on a sequence of previous values, but just on the one current value.

(In this implementation this algorithm can't learn anything!)

Procedure: 
- create a dictionary of all words in a given text
- store for each word all the direct following words
- if a word appears as input: lookup the word in the dictionary and choose one of the options

In [1]:
''' Libraries. '''

import numpy as np
import random
from nltk.tokenize import word_tokenize as tok
import string

## Tokenizer

The easiest way to tokenize a text is with the built-in function `split()`. But punctuation like .!, isn't separated from its preceding word. A more sophisticated solution is the usage of a library like nltk or spacy.

In [2]:
''' Read text and tokenize it with split(). '''
# Read text
with open('data/wiki_selection.txt', 'r') as f:
    text = f.read()
# Split into single words (tokens)
token = text.split()
print('Number of tokens:',len(token))
print(token[:50])

Number of tokens: 44032
['Aesthetics,', 'or', 'esthetics', '(),', 'is', 'a', 'branch', 'of', 'philosophy', 'that', 'deals', 'with', 'the', 'nature', 'of', 'beauty', 'and', 'taste,', 'as', 'well', 'as', 'the', 'philosophy', 'of', 'art', '(its', 'own', 'area', 'of', 'philosophy', 'that', 'comes', 'out', 'of', 'aesthetics).', 'It', 'examines', 'subjective', 'and', 'sensori-emotional', 'values,', 'or', 'sometimes', 'called', 'judgments', 'of', 'sentiment', 'and', 'taste.Aesthetics', 'covers']


In [3]:
''' Read text and tokenize it with NLTK. '''
token = tok(text)
print('Number of tokens:',len(token))
print(token[:50])

Number of tokens: 50415
['Aesthetics', ',', 'or', 'esthetics', '(', ')', ',', 'is', 'a', 'branch', 'of', 'philosophy', 'that', 'deals', 'with', 'the', 'nature', 'of', 'beauty', 'and', 'taste', ',', 'as', 'well', 'as', 'the', 'philosophy', 'of', 'art', '(', 'its', 'own', 'area', 'of', 'philosophy', 'that', 'comes', 'out', 'of', 'aesthetics', ')', '.', 'It', 'examines', 'subjective', 'and', 'sensori-emotional', 'values', ',', 'or']


## Vocabulary

Create pairs of tokens: one token as input and the preceding token as a possible output.

In [6]:
''' Create a generator with pairs of tokens. '''

def make_pairs(token):
    for i in range(len(token)- 1):
        yield (token[i], token[i+1:i+2])

pairs = make_pairs(token) # pairs is a generator object

In [5]:
''' Test pairs (execute the code above again after executing this one, otherwise the already printed pairs are gone.) '''

# for i in range(3):
#     print(next(iter(pairs)))

('Aesthetics', [','])
(',', ['or'])
('or', ['esthetics'])


In [7]:
''' Create a vocabulary of all tokens and map them to their preceding tokens. '''

# Create an empty dictionary.
vocabulary = {}

# Iterate through the pairs created above.
for current_token, next_token in pairs:
    # Check if the current token is already included into the dictionary.
    if current_token in vocabulary.keys():
        # If yes, append the next token to this entry.
        vocabulary[current_token].append(' '.join(next_token))
    else:
        # Otherwise create a new entry with the current token.
        vocabulary[current_token] = [' '.join(next_token)]

In [8]:
print('Size of the vocabulary:', len(vocabulary))

Size of the vocabulary: 6946


In [9]:
''' Inspect all options for a given token. '''
key = token[0]
print('All options for\n', key, ':', vocabulary[key])

All options for
 Aesthetics : [',', 'in', ',', 'and', 'is', 'encompasses', 'examines', 'is', ',', '.', 'and']


As you may see some tokens appear multiple times. This is an easy way to assure that the probabilty for that token is higher.

### Generate text

The following function takes a text sequence (or just one token) as input.<br>
It starts with the last token of the input and looks in the dictionary for all possible next tokens.<br>
Then one of this options is picked with the function `np.random.choice()`.

In [10]:
''' Function to generate n next token. '''

def generate_text(input_, n_token=1):
    # input_ = string of text. Arbitrary length, but only the last token is used for the prediction.
    # n_token = number of tokens that the function generates.
    
    # tokenize input
    gentext = tok(input_)
    
    # predict n_token new tokens
    for i in range(n_token):
        # token = last token of gentext
        token_inp = gentext[-1]
        # check if token is included into the dictionary
        if not token_inp in vocabulary.keys():
            # pick a random choice if not included
            token_inp = random.choice(list(vocabulary.keys()))
        # get all options for the last token of gentext
        # it is assumed that this token exists in the vocabulary
        options = vocabulary[token_inp]
        # choose one of this options
        choice = np.random.choice(options)
        # append it to the generated text
        gentext.append(choice)
    
    # create output
    output = ''
    # loop through all predicted tokens
    for token in gentext:
        # if token is a punctuation:
        if token in string.punctuation:
            # append it without a whitespace
            output += token
        else:
            # else add a whitespace before the token
            output += ' ' + token
    return output

In [13]:
''' Generated text. '''

# Pick a random input
key = token[random.randint(0, len(token))]
# Or use a string as input
key = 'The artistic code' 

generated_text = generate_text(key, 12)
print(generated_text)

 The artistic code, because there are the idea that they were a status of


In [14]:
''' Generate multiple texts at once to see the differences. '''
for i in range(5):
    print(generate_text(key, 15))

 The artistic code to the relations. An Introduction Defining science, W.V. If the present and
 The artistic code to train search for Drawing Caricaturas: Clarendon. Criticism, artificial neurons '' model
 The artistic code, such as well as a field of questions. Reinforcement learning: a sparse
 The artistic code to understand how facts and understand how they help us to enlarge their human intelligence
 The artistic code, as Abu Hamid Muhammad al-Ghazali that were among several distinctive explanations are countless subjects


### Sources

https://towardsdatascience.com/simulating-text-with-markov-chains-in-python-1a27e6d13fc6

https://mb-14.github.io/tech/2018/10/24/gomarkov.html